# CONTENT


[GENERAL_SCHEME](#GENERAL_SCHEME)<br>
[METRICS](#METRICS)<br>
[KEY_METRICS](#KEY_METRICS)<br>
[GROUPS_RANDOMIZATION](#GROUPS_RANDOMIZATION)<br>
[MDE](#MDE)<br>
[GROUP_SIZE](#GROUP_SIZE)<br>
[TEST_SELECTION](#TEST_SELECTION)<br>
[TESTS_DESCRIPTION](#TESTS_DESCRIPTION)<br>
[EXPERIMENT](#EXPERIMENT)<br>
[EXPERIMENT_VALIDATION](#EXPERIMENT_VALIDATION)<br>


[MDE](#MDE)<br>
[MDE](#MDE)<br>
[MDE](#MDE)<br>

[LINKS](#LINKS)

# GENERAL_SCHEME


[CONTENT](#CONTENT)<br>


## Основные этапы АБ теста: 
- выбираем основную и контрольные метрики
- формулируем гипотезы
- выбираем способ формирования выборки и ее параметры
- рассчитываем размер выборки и MDE
- проводим АА тест
- выбираем тип теста
- проведение эксперимента и сбор данных
- проверка валидности эксперимента
- расчет результатов и принятие решения


# METRICS

[CONTENT](#CONTENT)<br>

## Определяем метрики для тестирования:
- основную, в отношении которой проводим АБ тест
- контролируемые метрики, которые важны для функционировния и могут быть затронуты изменением прямо или косвенно

Основная цель выбора дополнительно контролируемых метрик - отследить возможные косвенные ухудшения функционирования продукта в результате внедрения изменений (например, стат значимо увеличиваем конверсию, но одновременно существенно падают просмотры).

## Преобразование метрик и альтернативные метрики

Если на этапе расчета MDE или длительности теста понимаем, что текущая метрика не позволяет провести тест в разумные сроки, размышляем над альтернативными метриками или над преобразованием текущих метрик.

### Возможные варианты альтернативных метрик:
- увеличить разброс значени таргетной метрики, с одной стороны это увеличит дисперсию. С другой стороны, размер отслеживаемого эффекта (который обратно квадратично влияет на объем необходимой выборки) существенно увеличится, что может повлечь существенное сокращение сроков проведения теста.
- вместо общего CTR считаем поюзерный CTR, но тут сталикаемся с проблемой большого количества пользователей с нулевыми целевыми действиям, нужно придавать пользователям вес и генерировать синтетическую метрику.
- меняем метрику на конверсионную, у которой дисперсия считается по распределению Бернулли и может быть кратно меньше изначальной.

# KEY_METRICS

[CONTENT](#CONTENT)<br>

#### ARPU - average revenue per user

$$
ARPU = \frac{Выручка}{\text{Количество пользователей}}
$$

#### ARPPU - average revenue per paying user

$$
ARPPU = \frac{Выручка}{\text{Количество платящих пользователей}}
$$


$$
ARPU = ARPPU * \text{Paying Share} = ARPPU * \text{СК}
$$

#### AOV - average order value

$$
AOV = \frac{Выручка}{\text{Количество заказов}}
$$

#### CTR - click through rate
Количество целевых действий (кликов) к показам

#### NMV - net merchandise value
Отношение количества кликов по рекламным объявлениям к количеству их показов

#### RR - retention rate (конверсионная метрика)
Процент пользователей, вернувшихся в приложение на n-ный день

#### CR - конверсия (Conversion Rate, CR)

$$
CR_{\text{ X to Y}} = \frac{K}{N} \text{* 100\%} = \frac{ARPU}{ARPPU}
$$
K - количество пользователей совершили целевое действие Y (или просто дошли до шага Y)<br>
N - количество пользователей совершили целевое действие X (или просто дошли до шага X)<br>


<br>При расчёте конверсий нам важен именно факт совершения пользователем действия, а не количество действий
<br>Если у нас есть бизнес убеждение, что ARPPU Постоянно, то мы, когда это удобно можем брать метрику CR вместо ARPU (по формуле выше она связана с конверсией через долю платящих пользователей)
<br>Часто дисперсия конверсии намного ниже, чем дисперсия самого ARPU, а значит нам нужно меньше данных (см. формулу) и можно провести менее времязатратный эксперимент.
<br>Однако, не стоит забывать, что ARPPU все же может измениться под влиянием фичи.

Дисперсия конверсионых метрик рассчитывает по распределению Бернулли
$$
Var_\text{Bernoulli} = \overline{p}\text{ * } (1 - \overline{p})
$$

Для примера используем z-тест
$$
z = \frac{\overline{p}_\text{test} - \overline{p}_\text{control}}{\sqrt{\frac{Var_\text{test}}{n_\text{test}}+\frac{Var_\text{control}}{n_\text{control}}}}
$$

# GROUPS_RANDOMIZATION

[CONTENT](#CONTENT)<br>

## Разделение на группы

<b> ⚠️ ВАЖНО

При планировании эксперимента нужно задуматься о потенциальном сетевом эффекте, то есть о ситуации, когда измененное поведение из treatment group может повлиять на поведение  control group

Вариант 1<br>
В самом простом случае (один эксперимент в единицу времени или достаточное количество данных для проведения нескольких экспериментов без пересечения пользователей) - выделяем простым случайным образом две одинаковые по размеру группы treatment и control 

## Предотвращение сетевого эффекта

###  в соц сетях и схожих продуктах (апример, маркетплейсах) можно разделить пользователей на непересекающиеся кластеры. 
Возможные способы разделения на кластеры
- соц графы (группы с минимальным количество связей),
- покупатели предпочитающие разные категори покупок
- ML алгоритмы (unsupervised learning)

### при широкой географии можно разделить по географическому признаку
- здесь важно отследить, чтобы группы были однородны по своей структуре, показателям метрик и поведению, иначе нарушится принцип репрезентативности групп

### switchback-тестирование
- делим пользователей по географии, но раскатываем фичу на опредленный период времени, далее раскатываем на другую и т.п. Возможность применения сильно зависит от типа отслеживаемых изменений, т.к. некоторые изменения могут иметь долговременный эффект на самого пользователя и его связи.

## Проверка разделения на группы

По итогам разбиения обязательно проводим проверку групп по основной метрике и контролируемым метрикам на предмет наличия стат значимых различий.

А именно, проводим стат тест(аналогичный тому, который будет использоваться в АБ тесте) и проверяем что количество ложных прокрашиваний (FPR - ошибок первого рода) не превышает α <br><br><br>


<b>При наличии достаточного количества предшествующих периодов, выбираем три равных периода:

- период теста (1)
- равный ему период непосредственно предшествующий тесту (2)
- еще один аналогичный период в прошлое (3)

Порядок действий в таком случае:
- в периоде 3 выделяем группы,
- в периоде 2 проверяем что тест не прокрашивается и данные "не расходятся"
- период 1 проводим тест



# MDE

[CONTENT](#CONTENT)<br>

# GROUP_SIZE

[CONTENT](#CONTENT)<br>

### Два основных варианта
- рассчет по формуле (см. ниже)
- монтекарлим реальные данные симулируя необходимый MDE на treatment и отслеживаем на каком объеме выборки получим прокрашивания соответствующие необходимой стат значимости.

#### Теоретическая формула расчета размера выборки n
$$
n = \frac{
(Var_{control} + Var_{test}) * (z_{a/2} + z_{b})^2}
{MDE^2} 
= \frac{
(Std_{control}^2 + Std_{test}^2) * (z_{a/2} + z_{b})^2}
{MDE^2} 
$$
где, 
$$
z_{a/2}  \text{   - квантиль стандартного нормального распределения, соответствующий половине ошибки первого рода}
$$ 
$$
z_{b} \text{ - квантиль стандартного нормального распределения, соответствующий вероятности ошибки второго рода}
$$ 

#### Пример расчета n методом Монте-Карло

# TEST_SELECTION

[CONTENT](#CONTENT)<br>

## Количественная метрика ARPU, ARPPU, etc
Используем Т-тест, бакетный тест. 
<br><br>Т-тест там где распредление близко к нормальному
<br>Бакетный используем там, где сильно ассиметричное распредление


## Конверсионная метрика CR

z-тест для пропорций

## Метрика отношений - AOV, кол-во товаров в корзине или их цена
Метрика отношений - метрика где единица анализа (в отношении чего направле тест) отличается от единицы рандомизации (критерий, по которому мы бьем на группы , например - пользователи)<br>
В таких метриках обычно не соблюдается принцип независимости наблюдений, следовательно анализировать разницу ошибки средних некорректно.

Бутсреп, линеаризация, дельта-метод

# TESTS_DESCRIPTION

[CONTENT](#CONTENT)<br>

# EXPERIMENT

[CONTENT](#CONTENT)<br>

#### Основные приницпы проведения эксперимента:
- подглядывать можно, но досрочно прерывать только в случае форс-мажора, то есть критически неприемлемое падение основной или контролируемых метрик
- в результате подглядывания могут, в том числе, выявиться проблемы со сбором, обработкой или записью данных
- при получении позитивных данных прерывать нельзя категорически. Возникает повышенный риск получения ошибки первого рода, то есть обнаружение стат. значимого эффекта там, где его нет (смещаем границу ошибки первого рода необоснованно к центру распределения при параметрических тестах).

In [2]:
2500/50000

0.05

# EXPERIMENT_VALIDATION

На что обратить внимание:
- проверяем, что пропорция между treatment и control сохранилась, как мы планировали на стадии разработки дизайна экмперимента (особенно актуально для конверсионных метрик, где часть пользователей могла просто не дойти до нужного этапа) 
- еще раз проводим АА-тест на пользователях/юнитах, которые реально попали в эксперимент. Метрики по предыдущему периоду не должны расходиться. 

# LINKS

[CONTENT](#CONTENT)<br>